# Some attacks from misuse of block ciphers

It's important that you use the right mode of operation for a block cipher. It's also important that when you use a mode of operation, it is set-up correctly.

## ECB - Electronic Code Book

We're going to look at the Electronic Code Book (ECB) for AES to kick us off. That mode of operation is never secure regardless of which block-cipher you pick.

We start by just demoing the general capability.

First we need to set-up a key.

In [2]:
from Crypto.Random import get_random_bytes

aes_key = get_random_bytes(16)


In [21]:
from Crypto.Cipher import AES

cipher = AES.new(aes_key, AES.MODE_ECB)

plaintext = b'Super Secret Msg'

cipherText = cipher.encrypt(plaintext)

print("Encrypted text:", cipherText)

plainText = cipher.decrypt(cipherText)

print("Recovered Plain Text:", plainText)



Encrypted text: b'$\xbb\x99\x99J\xe3\x84}m\xc8\x81*\x87C1\x8d'
Recovered Plain Text: b'Super Secret Msg'


So what's the problem here?

The problem is the same plain-text encrypts to the same cipher text!

Let's show this is insecure using the game!

In [4]:
class AES_ECBGameStrategy:
    def getMessages(self, trialNumber):
            if trialNumber == 0:
                m0 = b'Super Secret Msg'
                m1 = b'Super Secret Msg'
            else:
                m0 = b'Super Secret Msg'
                m1 = b'Super Urgent Msg'
            return m0, m1 

    def challenge(self, challenge, trialNumber):
            if (trialNumber == 0):
                self.savedMessage = challenge
            if self.savedMessage == challenge:
                return 0
            else:
                return 1

Next we need the rules of the game. We're going to fix one key across all iterations of the game!

In [5]:
class AESFixedKey:
    from Crypto.Random import get_random_bytes

    def __init__(self, mode, iv=get_random_bytes(16)):
       self.key = get_random_bytes(16)
       self.mode = mode
       self.iv = iv

    def encrypt(self, data):
        aes = AES.new(self.key, self.mode)
        if self.mode == AES.MODE_CBC:
            aes = AES.new(self.key, self.mode, iv=self.iv)
        return aes.encrypt(data)

    def decrypt(self, data):
        aes = AES.new(self.key, self.mode, iv=self.iv)
        return aes.decrypt(data)


And let's test it works how we want it to!

In [6]:
aesFixedKey = AESFixedKey(AES.MODE_ECB)
aesFixedKey.encrypt(plaintext)

b'\xcf\x8f\x16"\x1aG0\xc8+?\x08e\x8d\x02\x03\x0f'

Right, let's play the game!

In [7]:
from SemanticSecurityGame import SemanticSecurityGame

strategy = AES_ECBGameStrategy()
game = SemanticSecurityGame(aesFixedKey, strategy)

game.runGame(50)

{'Trials': 50, 'Wins': 49}

# Cipher Block Chaining Mode - CBC.

There is a security proof showing that if CBC is used *correctly* the strength of the scheme reduces to the underlying strength of the cipher.

However, if you don't follow the conditions of the proof (i.e. that the IV is completely random and never re-used) then terrible things happen. 

Let's explore!

## Reusing the same IV

Reusing the same IV between encryptions is bad, it basically behaves like our ECB example. Let's show why!

In [8]:
iv=b'0000000000000000'
aesFixedKey = AESFixedKey(AES.MODE_ECB, iv=iv)


Notice how the frame below always encrypts to the same output!

In [9]:

aesFixedKey.encrypt(b'Super Secret Msg')


b'\xaa\x88\xebn\xc9\xf6\xf2X\x97\xb0Z\xe3\xa4l9j'

## Attack: Repeated use of the same IV.

Notice, literally the same strategy as ECB works! No code modifications.

In [23]:
from SemanticSecurityGame import SemanticSecurityGame

strategy = AES_ECBGameStrategy()
game = SemanticSecurityGame(aesFixedKey, strategy)

game.runGame(50)

{'Trials': 50, 'Wins': 49}

## Attack: Predictable IVs

What happens if the IV is predictable? Let's work through this slowly!

In [11]:
iv=b'0000000000000000'
key=b'0000000000000000'

aes = AES.new(key,AES.MODE_CBC, iv=iv)
aes.encrypt(plaintext)

b'xJu\xa1\xf3\xb5\x85\xd9~\xbc/I\xd6c~\xc2'

So we learned with a fixed key and a fixed IV, it's the same value each time. How do we ensure that the IV is the same across multiple runs?

Suppose the IV is simply a counter. It counts up for each encryption!

In [12]:
iv=b'0000000000000001'
key=b'0000000000000000'

aes = AES.new(key,AES.MODE_CBC, iv=iv)
aes.encrypt(b'Super Secret Msf')

b'xJu\xa1\xf3\xb5\x85\xd9~\xbc/I\xd6c~\xc2'

Oops! It's the same problem with ECB. 

What happened is we cancelled out the increasing IV by decreasing the message by one "Msg" became "Msf" so the input block to CBC was the same in both cases. 

Let's build some attack code to demonstrate the concept.

In [36]:
class AES_CBC_GameStrategy:
    def getMessages(self, trialNumber):
            if trialNumber == 0:
                m0 = self.getIvValue(trialNumber)
                m1 = self.getIvValue(trialNumber)
            else:
                m0 = self.getIvValue(trialNumber)
                m1 = b'Super Urgent Msg'
            return m0, m1 

    def getIvValue(self, trialNumber):
         ivStr = ('0'*16 + str(trialNumber))[-16:]
         byteArray = bytearray()
         byteArray.extend(map(ord, ivStr))
         return byteArray
    
    def challenge(self, challenge, trialNumber):
            if (trialNumber == 0):
                self.savedMessage = challenge
            if self.savedMessage == challenge:
                return 0
            else:
                return 1

The idea here is that we know that the IV is Xor'ed with the plain-text.

We exploit this by simply sending the IV as the message, using the property:

$$ \forall A: A \oplus A = 0 $$

Thus we end up on the first trial getting basically an ECB chunk back which we can then force to be produced on every subsequent iteration.

In [38]:
class AES_CBC_Challenge:
    def __init__(self):
        from Crypto.Random import get_random_bytes
        self.key = get_random_bytes(16)
        self.iv = 0 
        
    def getIvValue(self): 
         # This is nasty, forgive the transgression.
         ivStr = ('0'*16 + str( self.iv))[-16:]
         byteArray = bytearray()
         byteArray.extend(map(ord, ivStr))
         return byteArray

    def encrypt(self, msg):
      aes = AES.new(self.key, AES.MODE_CBC, iv=self.getIvValue())
      self.iv = self.iv + 1
      return aes.encrypt(msg)

Let's see if it works! :)

In [42]:
from SemanticSecurityGame import SemanticSecurityGame

strategy = AES_CBC_GameStrategy()
challenge = AES_CBC_Challenge()
game = SemanticSecurityGame(challenge, strategy)

game.runGame(50)

{'Trials': 50, 'Wins': 50}